# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-647b0d393e-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Economy of EU States") 

Consider the following exploratory information need:

> Compare businesses across different sectors and types in E.U. countries

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wd:Q458`      | E.U.         | node |
| `wd:Q142`      | France       | node      |
| `wd:Q4830453`  | Business     | node      |
| `wd:Q6881511`  | Enterprise   | node      |
| `wd:Q29110228` | AXA          | node |
| `wd:Q43183`    | insurance    | node |




Also consider

```
{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
}



```

is the BGP to retrieve all **french enterprises and businesses**

## Workload Goals

1. Identify the BGP for obtaining the type, legal form, and industry of a company

2. Identify the BGP to retrieve all companies owned by a company located in a EU country

3. Which company has the largest presence in E.U.?

4. Companies have different 'legal forms', compare the number of companies divided in different legal forms

5. Analyze the number of companies per type, legal form, and industry in each state
 
   5.1 What are the top-3 legal form in E.U.? 
   
   5.2 For which companies is defined some form of income or market capitalization or total assets? What is the min, max, and average in each country for a given legal form?
   
   5.3 Which business in each country owns more businesses in other E.U. countries?
   
   5.4 What can we say about industry sectors in various countries?


In [1]:
# start your workflow here

In [2]:
queryString = """
SELECT COUNT(*)
WHERE { 

{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  . 
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
}
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '4577')]


1

<h3> TASK 1 STARTS HERE </h3>
Identify the BGP for obtaining the type, legal form, and industry of a company.

Get a closer look at AXA (wd:Q29110228).

In [6]:
#Q1
#get a closer look at AXA (wd:Q29110228)
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

wd:Q29110228 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1278'), ('pname', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1320'), ('pname', 'OpenCorporates ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('pname', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pname', 'owner of')]
[('p', 'http://www.wikidata.org/prop/direct/P213'), ('pname', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P2657'), ('pname', 'EU Transparency Register ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3500'), ('pname', 'Ringgold ID')]
[('p', 'http://www.wikidata.org/prop/direct/P355'), ('pname', 'subsidiary')]
[('p', 'http://www.wikidata.org/prop/direct/P3608'), ('pname', 'EU VAT number')]
[('p', 'http://www.wikidata.org/prop/direct/P4776'), ('pname', 'MAC Address Block Large ID')]
[('p', 'http://

17

Use P31 and P1454 (legal form) found above.

In [7]:
#Q2
#get a closer look at AXA (wd:Q29110228) using P31 and P1454 (legal form) found above
queryString = """
SELECT DISTINCT ?form ?fname ?o ?oname
WHERE { 

wd:Q29110228 wdt:P31 ?o;
             wdt:P1454 ?form.

#Labels
?o <http://schema.org/name> ?oname .
?form <http://schema.org/name> ?fname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('form', 'http://www.wikidata.org/entity/Q6832945'), ('fname', 'private company limited by shares'), ('o', 'http://www.wikidata.org/entity/Q4830453'), ('oname', 'business')]
[('form', 'http://www.wikidata.org/entity/Q6832945'), ('fname', 'private company limited by shares'), ('o', 'http://www.wikidata.org/entity/Q6881511'), ('oname', 'enterprise')]


2

Just use P31.

In [9]:
#Q3
#get a closer look at AXA (wd:Q29110228) using P31
queryString = """
SELECT ?o ?oname
WHERE { 

wd:Q29110228 wdt:P31 ?o.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4830453'), ('oname', 'business')]
[('o', 'http://www.wikidata.org/entity/Q6881511'), ('oname', 'enterprise')]


2

Check result from P355 (subsidiary).

In [10]:
#Q4
#check P355
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

wd:Q29110228 wdt:P355 ?o.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q41567544'), ('oname', 'Panasonic AVC Networks Czech')]


1

Use P749 (parent organization).

In [11]:
#Q5
#check P749
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

wd:Q29110228 wdt:P749 ?o.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q53247'), ('oname', 'Panasonic Corporation')]


1

Retrieve french assurance businesses.

In [12]:
#Q6
#try to use assurance uri
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?p wdt:P17 wd:Q142;
   wdt:P31 wd:Q43183. 

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q3331029'), ('pname', "Mutuelle d'assurance des instituteurs de France")]


1

Retrieve assurance businesses.

In [13]:
#Q7
#try to use assurance uri without france
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?p wdt:P31 wd:Q43183.

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q3331029'), ('pname', "Mutuelle d'assurance des instituteurs de France")]
[('p', 'http://www.wikidata.org/entity/Q7246539'), ('pname', 'Prize indemnity insurance')]
[('p', 'http://www.wikidata.org/entity/Q4443488'), ('pname', 'Business interruption insurance')]
[('p', 'http://www.wikidata.org/entity/Q2079582'), ('pname', 'bridge loan')]
[('p', 'http://www.wikidata.org/entity/Q137805'), ('pname', 'pension provider')]
[('p', 'http://www.wikidata.org/entity/Q811455'), ('pname', "builder's risk insurance")]
[('p', 'http://www.wikidata.org/entity/Q6899280'), ('pname', 'Money purchase')]
[('p', 'http://www.wikidata.org/entity/Q1735021'), ('pname', 'Comprehensive Cover')]
[('p', 'http://www.wikidata.org/entity/Q1538443'), ('pname', 'HBOS plc')]
[('p', 'http://www.wikidata.org/entity/Q5723049'), ('pname', 'Central Insurance of Iran')]
[('p', 'http://www.wikidata.org/entity/Q7907951'), ('pname', 'National Vaccine Injury Compensation Program')]
[('p

22

In [14]:
#Q8
#try to use assurance uri + legal form
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?p wdt:P31 wd:Q43183;
   wdt:P1454 ?form.

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q3331029'), ('pname', "Mutuelle d'assurance des instituteurs de France")]
[('p', 'http://www.wikidata.org/entity/Q1538443'), ('pname', 'HBOS plc')]


2

Retrieve some predicates of businesses.

In [16]:
#Q9
#find other business and check their predicates
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?s wdt:P31 wd:Q4830453;
   wdt:P1454 ?form;
   ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2486'), ('pname', 'FMD brand ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9279'), ('pname', 'Egapro gender equality index')]
[('p', 'http://www.wikidata.org/prop/direct/P2751'), ('pname', 'Roller Coaster Database ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7972'), ('pname', 'CVR person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7639'), ('pname', 'OverDrive publisher ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('pname', 'director / manager')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pname', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1128'), ('pname', 'employees')]
[('p', 'http://www.wikidata.org/prop/direct/P1167'), ('pnam

50

See what we can obtain using P101 (field of work).

In [17]:
#Q10
#use field of work (P101) found above
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

?s wdt:P31 wd:Q4830453;
   wdt:P1454 ?form;
   wdt:P17 wd:Q142;
   wdt:P101 ?o.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q978659'), ('oname', 'vehicle inspection')]
[('o', 'http://www.wikidata.org/entity/Q207652'), ('oname', 'chemical industry')]
[('o', 'http://www.wikidata.org/entity/Q1326624'), ('oname', 'electric utility')]
[('o', 'http://www.wikidata.org/entity/Q1384703'), ('oname', 'trade credit insurance')]
[('o', 'http://www.wikidata.org/entity/Q377688'), ('oname', 'type foundry')]
[('o', 'http://www.wikidata.org/entity/Q247478'), ('oname', 'security printing')]
[('o', 'http://www.wikidata.org/entity/Q13397636'), ('oname', 'street furniture')]
[('o', 'http://www.wikidata.org/entity/Q37038'), ('oname', 'advertising')]
[('o', 'http://www.wikidata.org/entity/Q2912428'), ('oname', 'ready-to-wear')]
[('o', 'http://www.wikidata.org/entity/Q1924856'), ('oname', 'metal construction')]
[('o', 'http://www.wikidata.org/entity/Q1155133'), ('oname', 'Email marketing')]
[('o', 'http://www.wikidata.org/entity/Q16861871'), ('oname', 'job training')]
[('o', 'http://ww

29

In [18]:
#Q11
#use (P176) found on Q9
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

?s wdt:P31 wd:Q4830453;
   wdt:P1454 ?form;
   wdt:P17 wd:Q142;
   wdt:P176 ?o.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
Empty


0

Retrieve some material or product produced using P1056 found in Q9.

In [19]:
#Q12
#use (P1056) found on Q9
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

?s wdt:P31 wd:Q4830453;
   wdt:P1454 ?form;
   wdt:P17 wd:Q142;
   wdt:P1056 ?o.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2671652'), ('oname', 'medical test')]
[('o', 'http://www.wikidata.org/entity/Q80831'), ('oname', 'integrated circuit')]
[('o', 'http://www.wikidata.org/entity/Q483639'), ('oname', 'cloud computing')]
[('o', 'http://www.wikidata.org/entity/Q161618'), ('oname', 'asymmetric digital subscriber line')]
[('o', 'http://www.wikidata.org/entity/Q12725'), ('oname', 'electricity')]
[('o', 'http://www.wikidata.org/entity/Q1420'), ('oname', 'motor car')]
[('o', 'http://www.wikidata.org/entity/Q185217'), ('oname', 'dairy product')]
[('o', 'http://www.wikidata.org/entity/Q11460'), ('oname', 'clothing')]
[('o', 'http://www.wikidata.org/entity/Q7889'), ('oname', 'video game')]
[('o', 'http://www.wikidata.org/entity/Q859281'), ('oname', 'wagon')]
[('o', 'http://www.wikidata.org/entity/Q2143661'), ('oname', 'corrective lens')]
[('o', 'http://www.wikidata.org/entity/Q93301'), ('oname', 'locomotive')]
[('o', 'http://www.wikidata.org/entity/Q81945'), ('oname',

50

Check how many french companies have P1056.

In [20]:
#Q13
#count how many french companies has P1056
queryString = """
SELECT (COUNT (DISTINCT(?p)) AS ?HowMany)
WHERE { 

{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  .
?p wdt:P1056 ?o.
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
?p wdt:P1056 ?o.
}
} 
"""

print("Results")
run_query(queryString)

Results
[('HowMany', '442')]


1

Same check but for P101.

In [21]:
#Q14
#count how many french companies has P101
queryString = """
SELECT (COUNT (DISTINCT(?p)) AS ?HowMany)
WHERE { 

{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  .
?p wdt:P101 ?o.
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
?p wdt:P101 ?o.
}
} 
"""

print("Results")
run_query(queryString)

Results
[('HowMany', '109')]


1

Retrieve properties of businesses but excluding literals.

In [22]:
#Q15
#as Q9 but filter literals
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?s wdt:P31 wd:Q4830453;
   wdt:P1454 ?form;
   ?p ?o.
   
FILTER(!isLiteral(?o))
#Labels

?p <http://schema.org/name> ?pname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('pname', 'director / manager')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pname', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pname', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('pname', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pname', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[

100

Count how many french companies has P452.

In [23]:
#Q16
#count how many french companies has P452
queryString = """
SELECT (COUNT (DISTINCT(?p)) AS ?HowMany)
WHERE { 

{ 
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q6881511  .
?p wdt:P452 ?o.
} UNION {
?p wdt:P17 wd:Q142  . 
?p wdt:P31 wd:Q4830453  . 
?p wdt:P452 ?o.
}
} 
"""

print("Results")
run_query(queryString)

Results
[('HowMany', '1573')]


1

Retrieve some objects of P452.

In [24]:
#Q17
#look at some results from P452
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

?s wdt:P31 wd:Q4830453;
   wdt:P1454 ?form;
   wdt:P17 wd:Q142;
   wdt:P452 ?o.

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q396622'), ('oname', 'agribusiness')]
[('o', 'http://www.wikidata.org/entity/Q126793'), ('oname', 'retail')]
[('o', 'http://www.wikidata.org/entity/Q44497'), ('oname', 'mining')]
[('o', 'http://www.wikidata.org/entity/Q2151621'), ('oname', 'energy industry')]
[('o', 'http://www.wikidata.org/entity/Q187939'), ('oname', 'manufacturing')]
[('o', 'http://www.wikidata.org/entity/Q207652'), ('oname', 'chemical industry')]
[('o', 'http://www.wikidata.org/entity/Q3510521'), ('oname', 'computer security')]
[('o', 'http://www.wikidata.org/entity/Q540912'), ('oname', 'food industry')]
[('o', 'http://www.wikidata.org/entity/Q29584635'), ('oname', 'Manufacture of railway locomotives and rolling stock')]
[('o', 'http://www.wikidata.org/entity/Q13405640'), ('oname', 'construction industry')]
[('o', 'http://www.wikidata.org/entity/Q2986369'), ('oname', 'semiconductor industry')]
[('o', 'http://www.wikidata.org/entity/Q56611700'), ('oname', 'internet indus

50

<h3> TASK 1 ENDS HERE </h3>
I found the following predicates P31 is for the type, P1454 for legal form, P452 for industry. On top of that i noticed that some companies have
#also P101 (field of work) and P1056 (product or material produced).

In [25]:
#TASK 1 ENDS HERE:
#i found the following predicates P31 is for the type, P1454 for legal form, P452 for industry. On top of that i noticed that some companies have
#also P101 (field of work) and P1056 (product or material produced)
queryString = """
SELECT DISTINCT ?c ?type ?form ?industry
WHERE { 

?c wdt:P31 ?type;
   wdt:P1454 ?form;
   wdt:P17 wd:Q142;
   wdt:P452 ?industry.

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q17177091'), ('type', 'http://www.wikidata.org/entity/Q4830453'), ('form', 'http://www.wikidata.org/entity/Q654502'), ('industry', 'http://www.wikidata.org/entity/Q540912')]
[('c', 'http://www.wikidata.org/entity/Q17177091'), ('type', 'http://www.wikidata.org/entity/Q6881511'), ('form', 'http://www.wikidata.org/entity/Q654502'), ('industry', 'http://www.wikidata.org/entity/Q540912')]
[('c', 'http://www.wikidata.org/entity/Q3036861'), ('type', 'http://www.wikidata.org/entity/Q4830453'), ('form', 'http://www.wikidata.org/entity/Q15091621'), ('industry', 'http://www.wikidata.org/entity/Q941594')]
[('c', 'http://www.wikidata.org/entity/Q3036861'), ('type', 'http://www.wikidata.org/entity/Q6881511'), ('form', 'http://www.wikidata.org/entity/Q15091621'), ('industry', 'http://www.wikidata.org/entity/Q941594')]
[('c', 'http://www.wikidata.org/entity/Q3036861'), ('type', 'http://www.wikidata.org/entity/Q210167'), ('form', 'http://www.wikidata.org/e

10

<h3>TASK 2 STARTS HERE</h3>
Identify the BGP to retrieve all companies owned by a company located in a EU country.

Use P1830 (owner of) found in Q1.

In [27]:
#Q18
#previously (Q1) i found "owner of" (P1830), let's use it on AXA
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

wd:Q29110228 wdt:P1830 ?o.

#Labels
?o <http://schema.org/name> ?oname .

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5446533'), ('oname', 'Ficosa')]
[('o', 'http://www.wikidata.org/entity/Q21178747'), ('oname', 'Younicos')]


2

In [28]:
#Q19
#closer look to previous result
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

wd:Q5446533 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location'), ('o', 'http://www.wikidata.org/entity/Q1492'), ('oname', 'Barcelona')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('pname', 'industry'), ('o', 'http://www.wikidata.org/entity/Q190117'), ('oname', 'automotive industry')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q29'), ('oname', 'Spain')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q4830453'), ('oname', 'business')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q6881511'), ('oname', 'enterprise')]


5

I think that P1830 is to find "sub-companies"  while P355/P749 is the reverse path (as seen in previous queries).

In [29]:
#Q20
#looking for class "company"
queryString = """
SELECT ?p ?pname ?o ?oname
WHERE { 

wd:Q6832945 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pname', 'applies to jurisdiction'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oname', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q145'), ('oname', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q134161'), ('oname', 'joint-stock company')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q155076'), ('oname', 'juridical person')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q18624259'), ('oname', 'private limited liability company')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q57655433'), ('oname', 'type of business entity in the UK')]


6

Iterate over P279.

In [31]:
#Q20
#so P1830 is to find "sub-companies"  while P355/P749 is the reverse path (as seen in previous queries)
#looking for class "company"
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

wd:Q6832945 wdt:P279* ?o.

#Labels
?o <http://schema.org/name> ?oname .

} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q830077'), ('oname', 'subject')]
[('o', 'http://www.wikidata.org/entity/Q6832945'), ('oname', 'private company limited by shares')]
[('o', 'http://www.wikidata.org/entity/Q1415187'), ('oname', 'economic agent')]
[('o', 'http://www.wikidata.org/entity/Q6671777'), ('oname', 'structure')]
[('o', 'http://www.wikidata.org/entity/Q679206'), ('oname', 'organizational unit')]
[('o', 'http://www.wikidata.org/entity/Q35120'), ('oname', 'entity')]
[('o', 'http://www.wikidata.org/entity/Q16334298'), ('oname', 'group of living things')]
[('o', 'http://www.wikidata.org/entity/Q43229'), ('oname', 'organization')]
[('o', 'http://www.wikidata.org/entity/Q4830453'), ('oname', 'business')]
[('o', 'http://www.wikidata.org/entity/Q6881511'), ('oname', 'enterprise')]
[('o', 'http://www.wikidata.org/entity/Q1589009'), ('oname', 'privately held company')]
[('o', 'http://www.wikidata.org/entity/Q16334295'), ('oname', 'group of humans')]
[('o', 'http://www.wikidata

40

From previous result I know that wd:Q783794 is for company. Count all the french companies.

In [32]:
#Q21
#try to count all french companies
queryString = """
SELECT (COUNT (DISTINCT ?p) AS ?Companies)
WHERE { 

?p wdt:P1454 wd:Q783794;
   wdt:P17 wd:Q142.

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('Companies', '35')]


1

Same above but iterate over P279.

In [33]:
#Q22
#wd:Q783794 is for company
#try to count all french companies, adding what found on Q20
queryString = """
SELECT (COUNT (DISTINCT ?p) AS ?Companies)
WHERE { 

?p wdt:P1454 ?form;
   wdt:P17 wd:Q142.
?form wdt:P279* wd:Q783794.

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('Companies', '2273')]


1

Try to find the BGP for being EU member.

In [34]:
#Q23
#find link with EU
queryString = """
SELECT ?p ?pname
WHERE { 

wd:Q142 ?p wd:Q458.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('pname', 'member of')]


1

Retrive EU business with highest number of subcompanies.

In [35]:
#Q24
#retrive eu business with highest number of subcompanies 
queryString = """
SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o. 
?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )
} 
ORDER BY DESC(?OwnedCompanies)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q381955'), ('OwnedCompanies', '109')]


1

In [36]:
#Q25
#check if exists a sort of sub-sub companies by iterating P1830
queryString = """
SELECT (COUNT(DISTINCT(?o)) AS ?Owned)
WHERE { 

wd:Q381955 wdt:P1830* ?o. 
}
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('Owned', '110')]


1

The following result has been edited some cells below because I notice an error.

In [37]:
queryString = """
SELECT ?s ?sname ?o ?oname
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o. 
?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )

#Labels
?s <http://schema.org/name> ?sname .
?o <http://schema.org/name> ?oname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q492820'), ('sname', 'Salling Group'), ('o', 'http://www.wikidata.org/entity/Q18162020'), ('oname', 'Netto UK')]
[('s', 'http://www.wikidata.org/entity/Q28871958'), ('sname', 'Seier Capital Denmark'), ('o', 'http://www.wikidata.org/entity/Q15573853'), ('oname', 'Geranium')]
[('s', 'http://www.wikidata.org/entity/Q23639837'), ('sname', 'Mila'), ('o', 'http://www.wikidata.org/entity/Q23638508'), ('oname', 'ArtPeople Holding')]
[('s', 'http://www.wikidata.org/entity/Q635484'), ('sname', 'Armani'), ('o', 'http://www.wikidata.org/entity/Q16586266'), ('oname', 'Palazzo Orsini')]
[('s', 'http://www.wikidata.org/entity/Q493854'), ('sname', 'SSE plc'), ('o', 'http://www.wikidata.org/entity/Q4877211'), ('oname', 'Beatrice Wind Farm')]
[('s', 'http://www.wikidata.org/entity/Q493854'), ('sname', 'SSE plc'), ('o', 'http://www.wikidata.org/entity/Q820613'), ('oname', 'Greater Gabbard wind farm')]
[('s', 'http://www.wikidata.org/entity/Q524989'), ('sname

10

<h3> TASK 3 STARTS HERE </h3>
Which company has the largest presence in EU.?

In [39]:
#Q26
queryString = """
SELECT ?s ?sname WHERE
{
{SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o. 
?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )
} 
ORDER BY DESC(?OwnedCompanies)
LIMIT 1
}

#Labels
?s <http://schema.org/name> ?sname .
}
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q381955'), ('sname', 'Azienda Trasporti Milanesi')]


1

As above but iterating over P1830.

In [40]:
#Q27
#as above but iterating over P1830
queryString = """
SELECT ?s ?sname ?OwnedCompanies WHERE
{
{SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830* ?o. 
?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )
} 
ORDER BY DESC(?OwnedCompanies)
LIMIT 1
}

#Labels
?s <http://schema.org/name> ?sname .
}
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q846484'), ('sname', 'Caisse des dépôts et consignations'), ('OwnedCompanies', '151')]


1

Check results removing type.

In [41]:
#Q28
#check results removing type
queryString = """
SELECT ?s ?sname ?OwnedCompanies WHERE
{
{SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P1454 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o. 
?country wdt:P463 wd:Q458.

} 
ORDER BY DESC(?OwnedCompanies)
LIMIT 1
}

#Labels
?s <http://schema.org/name> ?sname .
}
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q513666'), ('sname', 'German Alpine Club'), ('OwnedCompanies', '170')]


1

In [43]:
#Q29
queryString = """
SELECT ?o ?oname WHERE
{

wd:Q513666 wdt:P1830 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1735212'), ('oname', 'Kasseler Hütte')]
[('o', 'http://www.wikidata.org/entity/Q2550679'), ('oname', 'Wasseralm')]
[('o', 'http://www.wikidata.org/entity/Q19964854'), ('oname', 'Taubensteinhaus')]
[('o', 'http://www.wikidata.org/entity/Q2246287'), ('oname', 'Schmidt-Zabierow-Hütte')]
[('o', 'http://www.wikidata.org/entity/Q2461881'), ('oname', 'Tutzinger Hütte')]
[('o', 'http://www.wikidata.org/entity/Q1803733'), ('oname', 'Landsberger Hütte')]
[('o', 'http://www.wikidata.org/entity/Q187379'), ('oname', 'Gaudeamus Hut')]
[('o', 'http://www.wikidata.org/entity/Q1350220'), ('oname', 'Erfurter Hütte')]
[('o', 'http://www.wikidata.org/entity/Q1640756'), ('oname', 'Rastkogelhütte')]
[('o', 'http://www.wikidata.org/entity/Q1643003'), ('oname', 'Watzmannhaus')]
[('o', 'http://www.wikidata.org/entity/Q2151300'), ('oname', 'Richterhütte')]
[('o', 'http://www.wikidata.org/entity/Q2169123'), ('oname', 'Rotwandhaus')]
[('o', 'http://www.wikidata.org/

25

I think that removing type "business" led to include entities which are not company. i m not sure that Azienda Trasporti owns companies, let's check.

In [44]:
#Q30
#removing type "business" led to include entities which are not company 
#i m not sure that Azienda Trasporti owns companies, let's check
queryString = """
SELECT ?o ?oname WHERE
{

wd:Q381955 wdt:P1830 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2815996'), ('oname', 'De Angeli metro station')]
[('o', 'http://www.wikidata.org/entity/Q3094926'), ('oname', 'Gambara metro station')]
[('o', 'http://www.wikidata.org/entity/Q3565090'), ('oname', 'Wagner metro station')]
[('o', 'http://www.wikidata.org/entity/Q3559341'), ('oname', 'Vimodrone metro station')]
[('o', 'http://www.wikidata.org/entity/Q2904609'), ('oname', 'Bisceglie metro station')]
[('o', 'http://www.wikidata.org/entity/Q3150918'), ('oname', 'Inganni metro station')]
[('o', 'http://www.wikidata.org/entity/Q3403179'), ('oname', 'Primaticcio metro station')]
[('o', 'http://www.wikidata.org/entity/Q1087026'), ('oname', 'Garibaldi FS metro station')]
[('o', 'http://www.wikidata.org/entity/Q1087090'), ('oname', 'Lampugnano metro station')]
[('o', 'http://www.wikidata.org/entity/Q1087114'), ('oname', 'Moscova metro station')]
[('o', 'http://www.wikidata.org/entity/Q1087580'), ('oname', 'Sant’Ambrogio metro station')]
[('o', 'http

25

Indeed they look like bus stops..

In [45]:
#Q31
#they look like bus stops, wd:Q1087026
queryString = """
SELECT ?o ?oname WHERE
{

wd:Q1087026 wdt:P31 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q22808403'), ('oname', 'station located underground')]
[('o', 'http://www.wikidata.org/entity/Q928830'), ('oname', 'metro station')]


2

Since "owner of" is intended in a larger sense, i have to edit the previous queries as follows.

In [47]:
#Q32
#since "owner of" is intended in a larger sense, i have to edit the previous queries as follows:
queryString = """
SELECT DISTINCT ?o ?oname
WHERE { 

?s wdt:P1454 ?o. 

#Labels
?o <http://schema.org/name> ?oname .
} 
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q17002660'), ('oname', 'fan translation')]
[('o', 'http://www.wikidata.org/entity/Q8035059'), ('oname', 'workplace democracy')]
[('o', 'http://www.wikidata.org/entity/Q11799049'), ('oname', 'public institution')]
[('o', 'http://www.wikidata.org/entity/Q20746571'), ('oname', 'public company limited by shares')]
[('o', 'http://www.wikidata.org/entity/Q60836687'), ('oname', 'associazione sportiva dilettantistica')]
[('o', 'http://www.wikidata.org/entity/Q14635346'), ('oname', 'multi-channel network')]
[('o', 'http://www.wikidata.org/entity/Q3639678'), ('oname', 'circulating library')]
[('o', 'http://www.wikidata.org/entity/Q18340464'), ('oname', 'sociedade anónima')]
[('o', 'http://www.wikidata.org/entity/Q2624661'), ('oname', 'Sp. z o.o.')]
[('o', 'http://www.wikidata.org/entity/Q375336'), ('oname', 'film studio')]
[('o', 'http://www.wikidata.org/entity/Q938236'), ('oname', 'panel')]
[('o', 'http://www.wikidata.org/entity/Q108514063'), ('ona

25

<h3> TASK 2 ENDS HERE </h3>
This is the BGP to retrieve all companies owned by a company located in a EU country.

In [48]:
#TASK 2 EDITED
queryString = """
SELECT ?s ?sname ?o ?oname
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o. 
?o wdt:P31 ?type.
?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )

#Labels
?s <http://schema.org/name> ?sname .
?o <http://schema.org/name> ?oname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q4162216'), ('sname', 'Energetický a průmyslový holding'), ('o', 'http://www.wikidata.org/entity/Q46965053'), ('oname', 'LEAG Holding')]
[('s', 'http://www.wikidata.org/entity/Q4162216'), ('sname', 'Energetický a průmyslový holding'), ('o', 'http://www.wikidata.org/entity/Q27485698'), ('oname', 'Slovak Power Holding')]
[('s', 'http://www.wikidata.org/entity/Q4162216'), ('sname', 'Energetický a průmyslový holding'), ('o', 'http://www.wikidata.org/entity/Q11828362'), ('oname', 'PG Silesia')]
[('s', 'http://www.wikidata.org/entity/Q4162216'), ('sname', 'Energetický a průmyslový holding'), ('o', 'http://www.wikidata.org/entity/Q29650125'), ('oname', 'BAULIGA')]
[('s', 'http://www.wikidata.org/entity/Q41365932'), ('sname', 'AGEL'), ('o', 'http://www.wikidata.org/entity/Q58218626'), ('oname', 'Avenier')]
[('s', 'http://www.wikidata.org/entity/Q673327'), ('sname', 'Arkema'), ('o', 'http://www.wikidata.org/entity/Q2074255'), ('oname', 'Bostik')]
[

10

Continue TASK 3, retrieve the EU business who onws the highest number of companies.

In [49]:
#TASK 3 
queryString = """
SELECT ?s ?sname ?OwnedCompanies WHERE
{
{SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830* ?o.
?o wdt:P31 ?type.
?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )
} 
ORDER BY DESC(?OwnedCompanies)
LIMIT 1
}

#Labels
?s <http://schema.org/name> ?sname .
}
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q846484'), ('sname', 'Caisse des dépôts et consignations'), ('OwnedCompanies', '58')]


1

In [50]:
#Q33
#closer look to that entity
queryString = """
SELECT ?o ?oname WHERE
{

wd:Q846484 wdt:P1830 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1127887'), ('oname', 'Vivendi')]
[('o', 'http://www.wikidata.org/entity/Q1121170'), ('oname', 'Compagnie Nationale du Rhône')]
[('o', 'http://www.wikidata.org/entity/Q2362964'), ('oname', 'Bpifrance')]
[('o', 'http://www.wikidata.org/entity/Q442637'), ('oname', 'Transdev')]
[('o', 'http://www.wikidata.org/entity/Q1632461'), ('oname', 'Veolia')]
[('o', 'http://www.wikidata.org/entity/Q373724'), ('oname', 'La Poste')]
[('o', 'http://www.wikidata.org/entity/Q726531'), ('oname', 'Théâtre des Champs-Élysées')]
[('o', 'http://www.wikidata.org/entity/Q1023930'), ('oname', 'CNP Assurances')]
[('o', 'http://www.wikidata.org/entity/Q3147573'), ('oname', 'Icade')]
[('o', 'http://www.wikidata.org/entity/Q3117552'), ('oname', 'CDC Habitat')]
[('o', 'http://www.wikidata.org/entity/Q624378'), ('oname', 'Compagnie des Alpes')]
[('o', 'http://www.wikidata.org/entity/Q286494'), ('oname', 'Quick')]


12

Retrieve the EU business which onws the highest number of companies but not considering the "sub-sub" companies (so no iteration over 1830).

In [51]:
#TASK 3 without iteration on P1830
queryString = """
SELECT ?s ?sname ?OwnedCompanies WHERE
{
{SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o.
?o wdt:P31 ?type.
?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )
} 
ORDER BY DESC(?OwnedCompanies)
LIMIT 1
}

#Labels
?s <http://schema.org/name> ?sname .
}
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q504998'), ('sname', 'LVMH'), ('OwnedCompanies', '23')]


1

In [52]:
#Q34
#closer look to that entity
queryString = """
SELECT ?o ?oname WHERE
{

wd:Q504998 wdt:P1830 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q895795'), ('oname', 'Kenzo')]
[('o', 'http://www.wikidata.org/entity/Q699529'), ('oname', 'Moët & Chandon')]
[('o', 'http://www.wikidata.org/entity/Q923193'), ('oname', 'Les Échos')]
[('o', 'http://www.wikidata.org/entity/Q966905'), ('oname', 'Loewe')]
[('o', 'http://www.wikidata.org/entity/Q191485'), ('oname', 'Louis Vuitton')]
[('o', 'http://www.wikidata.org/entity/Q926575'), ('oname', 'Fendi')]
[('o', 'http://www.wikidata.org/entity/Q142348'), ('oname', 'Le Parisien')]
[('o', 'http://www.wikidata.org/entity/Q1553251'), ('oname', 'Guerlain')]
[('o', 'http://www.wikidata.org/entity/Q1200698'), ('oname', 'Le Bon Marché')]
[('o', 'http://www.wikidata.org/entity/Q26196912'), ('oname', 'Cheval Blanc St-Barth Isle de France')]
[('o', 'http://www.wikidata.org/entity/Q274'), ('oname', 'Ardbeg')]
[('o', 'http://www.wikidata.org/entity/Q2056628'), ('oname', 'Acqua di Parma')]
[('o', 'http://www.wikidata.org/entity/Q821969'), ('oname', 'Berluti')]

25

<h3> TASK 3 ENDS HERE </h3>
I found out that LVMH is the company which owns "directly" the highest number of other companies.
Meanwhile if we consider also the companies owned by an owned company the n.1 is Caisse des dépôts et consignations.

<h3> TASK 4 STARTS HERE </h3>
Compare the number of companies divided in different legal forms.

Use P1454 (legal form) found in TASK 1. Retrieve the most common legal form for EU companies.

In [55]:
#Q35
#try to use P1454 
queryString = """
SELECT ?legal ?lname ?HowMany WHERE
{
{SELECT ?legal (COUNT(DISTINCT (?s)) AS ?HowMany)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1454 ?legal.

?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )
} 
GROUP BY (?legal)
}

#Labels
?legal <http://schema.org/name> ?lname .
}
ORDER BY DESC(?HowMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('legal', 'http://www.wikidata.org/entity/Q15646299'), ('lname', 'společnost s ručením omezeným'), ('HowMany', '13073')]
[('legal', 'http://www.wikidata.org/entity/Q3742494'), ('lname', 'akciová společnost'), ('HowMany', '5546')]
[('legal', 'http://www.wikidata.org/entity/Q460178'), ('lname', 'GmbH'), ('HowMany', '2898')]
[('legal', 'http://www.wikidata.org/entity/Q134161'), ('lname', 'joint-stock company'), ('HowMany', '2599')]
[('legal', 'http://www.wikidata.org/entity/Q15649019'), ('lname', 'Spoločnosť s ručením obmedzeným'), ('HowMany', '2306')]
[('legal', 'http://www.wikidata.org/entity/Q897103'), ('lname', 'GmbH & Co. KG'), ('HowMany', '1145')]
[('legal', 'http://www.wikidata.org/entity/Q77872838'), ('lname', 'akciová spoločnosť'), ('HowMany', '837')]
[('legal', 'http://www.wikidata.org/entity/Q422062'), ('lname', 'Aktiengesellschaft'), ('HowMany', '778')]
[('legal', 'http://www.wikidata.org/entity/Q333920'), ('lname', 'Oy'), ('HowMany', '601')]
[('legal', 'http://www.wi

10

In [56]:
#Q36
#check results
queryString = """
SELECT ?o ?oname WHERE
{

wd:Q15646299 wdt:P31 ?o.

#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q67304002'), ('oname', 'type of business entity in Czech Republic')]


1

<h3> TASK 5.1 HERE </h3>
Retrieve the top 3 legal forms in Europe.

In [2]:
#make legal forms more readable using labels 
queryString = """
SELECT ?legal ?lname ?HowMany ?tname WHERE
{
?legal wdt:P31 ?type.

{SELECT ?legal (COUNT(DISTINCT (?s)) AS ?HowMany)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1454 ?legal.

?country wdt:P463 wd:Q458.

FILTER( (?type=wd:Q6881511)||(?type=wd:Q4830453) )
} 
GROUP BY (?legal)
}

#Labels
?legal <http://schema.org/name> ?lname .
?type <http://schema.org/name> ?tname .
}
ORDER BY DESC(?HowMany)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('legal', 'http://www.wikidata.org/entity/Q15646299'), ('lname', 'společnost s ručením omezeným'), ('HowMany', '13073'), ('tname', 'type of business entity in Czech Republic')]
[('legal', 'http://www.wikidata.org/entity/Q3742494'), ('lname', 'akciová společnost'), ('HowMany', '5546'), ('tname', 'type of business entity in Czech Republic')]
[('legal', 'http://www.wikidata.org/entity/Q460178'), ('lname', 'GmbH'), ('HowMany', '2898'), ('tname', 'type of legal entity in Germany')]


3

<h3> TASK 5.2 STARTS HERE </h3>
For which companies is defined some form of income or market capitalization or total assets?
What is the min, max, and average in each country for a given legal form?

Look for form of income, market cap or assets using LMVH (found during task 3)

In [6]:
#Q37
#look for form of income, market cap or assets using LMVH (found during task 3)
queryString = """
SELECT DISTINCT ?p ?pname WHERE
{

wd:Q504998 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1037'), ('pname', 'director / manager')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('pname', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('pname', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1128'), ('pname', 'employees')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('pname', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1278'), ('pname', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1320'), ('pname', 'OpenCorporates ID')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('pname', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pname', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headqu

50

Include only results with literals since we are looking for numbers.

In [8]:
#Q38
#look for form of income, market cap or assets using LMVH (found during task 3)
#filter literal because i m looking for numbers
queryString = """
SELECT DISTINCT ?p ?pname WHERE
{

wd:Q504998 ?p ?o.

FILTER (isLiteral(?o))

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1128'), ('pname', 'employees')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('pname', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1278'), ('pname', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1320'), ('pname', 'OpenCorporates ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1616'), ('pname', 'SIREN number')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pname', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P2003'), ('pname', 'Instagram username')]
[('p', 'http://www.wikidata.org/prop/direct/P2013'), ('pname', 'Facebook ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2088'), ('pname', 'Crunchbase organization ID')]
[('p', 'http://www.wikidata.org/prop/direct/P213'), ('pname', 'ISNI')]
[('p', 'http://www.wikidata.org/prop/direct/P2139'), ('pname', 'tot

38

Above i found P2295 (net profit), P2139 (total revenue).

In [9]:
#Q39
#above i found P2295 (net profit), P2139 (total revenue)
queryString = """
SELECT DISTINCT ?net ?revenue WHERE
{

wd:Q504998 wdt:P2295 ?net;
           wdt:P2139 ?revenue.

}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('net', '3319000000'), ('revenue', '53700000000')]


1

Retrieve some properites of a french business.

In [10]:
#Q40
#run Q38 on a general french business
queryString = """
SELECT DISTINCT ?p ?pname WHERE
{

?s wdt:P17 wd:Q142;
   wdt:P31 wd:Q4830453  . 
?s ?p ?o.

FILTER (isLiteral(?o))

#Labels
?p <http://schema.org/name> ?pname .
}
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9279'), ('pname', 'Egapro gender equality index')]
[('p', 'http://www.wikidata.org/prop/direct/P3281'), ('pname', 'French National Assembly Lobbyist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2133'), ('pname', 'total debt')]
[('p', 'http://www.wikidata.org/prop/direct/P4015'), ('pname', 'Vimeo identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1128'), ('pname', 'employees')]
[('p', 'http://www.wikidata.org/prop/direct/P1167'), ('pname', 'USB vendor ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('pname', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1278'), ('pname', 'Legal Entity Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1315'), ('pname', 'NLA Trove ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1320'), ('pname', 'OpenCorporates ID')]
[('p', 'http://www.wikidata.org/prop/direct

50

Check if top three results of task 5.1 are restricted to that countries.

In [11]:
#Q42
#check if top three results of task 5.1 are restricted to that countries
queryString = """
SELECT (COUNT(DISTINCT (?s)) AS ?HowMany)
WHERE { 

?s wdt:P17 ?country;
   wdt:P1454 wd:Q460178.

FILTER(?country!=wd:Q183) #germany
}
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('HowMany', '176')]


1

Most of them yes, so i pick the fourth result of Q36 (wd:Q134161 joint-stock company).Count how many of them have the 4 properties found above, then count for each single property to see which one is the best to use.

In [12]:
#Q43
#most of them yes, so i pick the fourth result of Q36 (wd:Q134161 joint-stock company) 
#check how many have all the 4 Property found in Q38 and Q40
queryString = """
SELECT (COUNT(DISTINCT (?s)) AS ?HowMany)
WHERE { 

?s wdt:P17 ?country;
   wdt:P1454 wd:Q134161;
   wdt:P2295 ?net;
   wdt:P2139 ?revenue;
   wdt:P2226 ?mcap;
   wdt:P2403 ?assets.
   
?country wdt:P463 wd:Q458. #eu
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('HowMany', '0')]


1

In [13]:
#Q44
#repeat only for market cap and assets
queryString = """
SELECT (COUNT(DISTINCT (?s)) AS ?HowMany)
WHERE { 

?s wdt:P17 ?country;
   wdt:P1454 wd:Q134161;
   wdt:P2226 ?mcap;
   wdt:P2403 ?assets.
   
?country wdt:P463 wd:Q458. #eu
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('HowMany', '0')]


1

In [14]:
#Q45
#repeat with generic legal form
queryString = """
SELECT (COUNT(DISTINCT (?s)) AS ?HowMany)
WHERE { 

?s wdt:P17 ?country;
   wd:P1454 ?form;
   wdt:P2226 ?mcap;
   wdt:P2403 ?assets.
   
?country wdt:P463 wd:Q458. #eu
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('HowMany', '0')]


1

In [15]:
#Q46
#check how many entities have P2226
queryString = """
SELECT (COUNT(DISTINCT (?s)) AS ?HowMany)
WHERE { 

?s wdt:P2226 ?mcap.
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('HowMany', '203')]


1

In [16]:
#Q47
#check how many entities have P2295
queryString = """
SELECT (COUNT(DISTINCT (?s)))
WHERE { 

?s wdt:P2295 ?net.
#?s wdt:P2139 ?revenue.
#?s wdt:P2403 ?assets.
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2308')]


1

In [17]:
#Q48
#check how many entities have P2139
queryString = """
SELECT (COUNT(DISTINCT (?s)))
WHERE { 

#?s wdt:P2295 ?net.
?s wdt:P2139 ?revenue.
#?s wdt:P2403 ?assets.
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '13424')]


1

In [18]:
#Q49
#check how many entities have P2403
queryString = """
SELECT (COUNT(DISTINCT (?s)))
WHERE { 

#?s wdt:P2295 ?net.
#?s wdt:P2139 ?revenue.
?s wdt:P2403 ?assets.
}
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1903')]


1

In [22]:
#Q50
#repeat the four queries above and group by legal form
queryString = """
SELECT ?legal (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P2295 ?net;
   wdt:P1454 ?legal.
}
GROUP BY (?legal)
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('legal', 'http://www.wikidata.org/entity/Q3742494'), ('howMany', '422')]
[('legal', 'http://www.wikidata.org/entity/Q134161'), ('howMany', '317')]
[('legal', 'http://www.wikidata.org/entity/Q15646299'), ('howMany', '286')]
[('legal', 'http://www.wikidata.org/entity/Q891723'), ('howMany', '176')]
[('legal', 'http://www.wikidata.org/entity/Q13641190'), ('howMany', '125')]
[('legal', 'http://www.wikidata.org/entity/Q1589009'), ('howMany', '40')]
[('legal', 'http://www.wikidata.org/entity/Q166280'), ('howMany', '39')]
[('legal', 'http://www.wikidata.org/entity/Q67207116'), ('howMany', '32')]
[('legal', 'http://www.wikidata.org/entity/Q1480166'), ('howMany', '24')]
[('legal', 'http://www.wikidata.org/entity/Q21191682'), ('howMany', '22')]


10

In [23]:
#Q51
#repeat the four queries above and group by legal form
queryString = """
SELECT ?legal (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P2139 ?revenue;
   wdt:P1454 ?legal.
}
GROUP BY (?legal)
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('legal', 'http://www.wikidata.org/entity/Q134161'), ('howMany', '552')]
[('legal', 'http://www.wikidata.org/entity/Q3742494'), ('howMany', '419')]
[('legal', 'http://www.wikidata.org/entity/Q891723'), ('howMany', '408')]
[('legal', 'http://www.wikidata.org/entity/Q15646299'), ('howMany', '304')]
[('legal', 'http://www.wikidata.org/entity/Q9299236'), ('howMany', '224')]
[('legal', 'http://www.wikidata.org/entity/Q13641190'), ('howMany', '137')]
[('legal', 'http://www.wikidata.org/entity/Q460178'), ('howMany', '97')]
[('legal', 'http://www.wikidata.org/entity/Q1500859'), ('howMany', '88')]
[('legal', 'http://www.wikidata.org/entity/Q18631225'), ('howMany', '83')]
[('legal', 'http://www.wikidata.org/entity/Q1589009'), ('howMany', '80')]


10

In [25]:
#Q52
#repeat the four queries above and group by legal form
queryString = """
SELECT ?legal (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P2403 ?assets;
   wdt:P1454 ?legal.
}
GROUP BY (?legal)
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('legal', 'http://www.wikidata.org/entity/Q3742494'), ('howMany', '467')]
[('legal', 'http://www.wikidata.org/entity/Q134161'), ('howMany', '337')]
[('legal', 'http://www.wikidata.org/entity/Q15646299'), ('howMany', '310')]
[('legal', 'http://www.wikidata.org/entity/Q891723'), ('howMany', '263')]
[('legal', 'http://www.wikidata.org/entity/Q13641190'), ('howMany', '66')]
[('legal', 'http://www.wikidata.org/entity/Q9299236'), ('howMany', '59')]
[('legal', 'http://www.wikidata.org/entity/Q67207116'), ('howMany', '40')]
[('legal', 'http://www.wikidata.org/entity/Q18631225'), ('howMany', '36')]
[('legal', 'http://www.wikidata.org/entity/Q422062'), ('howMany', '31')]
[('legal', 'http://www.wikidata.org/entity/Q88537331'), ('howMany', '20')]


10

In [26]:
#Q53
#repeat the four queries above and group by legal form
queryString = """
SELECT ?legal (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P2226 ?mcap;
   wdt:P1454 ?legal.
}
GROUP BY (?legal)
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('legal', 'http://www.wikidata.org/entity/Q891723'), ('howMany', '66')]
[('legal', 'http://www.wikidata.org/entity/Q134161'), ('howMany', '32')]
[('legal', 'http://www.wikidata.org/entity/Q88537331'), ('howMany', '12')]
[('legal', 'http://www.wikidata.org/entity/Q422062'), ('howMany', '8')]
[('legal', 'http://www.wikidata.org/entity/Q13641190'), ('howMany', '8')]
[('legal', 'http://www.wikidata.org/entity/Q1436139'), ('howMany', '7')]
[('legal', 'http://www.wikidata.org/entity/Q57655560'), ('howMany', '7')]
[('legal', 'http://www.wikidata.org/entity/Q18195767'), ('howMany', '6')]
[('legal', 'http://www.wikidata.org/entity/Q3742494'), ('howMany', '5')]
[('legal', 'http://www.wikidata.org/entity/Q1589009'), ('howMany', '5')]


10

Retrieve min market cap for wd:Q3742494 grouped by country.

In [29]:
#Q54
#retrieve min market cap for wd:Q3742494 grouped by country
queryString = """
SELECT ?country (MIN(?cap) AS ?min)
WHERE { 

?s wdt:P2226 ?cap;
   wdt:P1454 wd:Q3742494;
   wdt:P17 ?country.
   
FILTER(?cap>0)
}
GROUP BY (?country)
ORDER BY ASC(?min)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q213'), ('min', '7100000000')]


1

wd:Q3742494 refers to Czech as i found out in task 3 so it's logical just one result. Run the same query but using the second result of Q53.

In [30]:
#Q55
#wd:Q3742494 refers to Czech as i found out in task 3 so it's logical just one result
#try on second result of Q53
queryString = """
SELECT ?country (MIN(?cap) AS ?min)
WHERE { 

?s wdt:P2226 ?cap;
   wdt:P1454 wd:Q134161;
   wdt:P17 ?country.
   
FILTER(?cap>0)
}
GROUP BY (?country)
ORDER BY ASC(?min)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q39'), ('min', '39000000')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('min', '110300000')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('min', '200804100')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('min', '200804100')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('min', '486000000000')]
[('country', 'http://www.wikidata.org/entity/Q851'), ('min', '2458000000000')]


6

Get max of market cap for some countries.

In [46]:
#Q56
#get max
queryString = """
SELECT ?country (MAX(?cap) AS ?max)
WHERE { 

?s wdt:P2226 ?cap;
   wdt:P1454 wd:Q134161;
   wdt:P17 ?country.
   
FILTER(?cap>0)
}
GROUP BY (?country)
ORDER BY DESC(?max)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q851'), ('max', '2458000000000')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('max', '789000000000')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('max', '650340000000')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('max', '486000000000')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('max', '200804100')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('max', '39000000')]


6

Compute the average market cap.

In [44]:
#Q57
#get avg and labels
queryString = """
SELECT ?country ?cname ?avg WHERE
{
{SELECT ?country (AVG(?cap) AS ?avg)
WHERE { 

?s wdt:P2226 ?cap;
   wdt:P1454 wd:Q134161;
   wdt:P17 ?country.
   
FILTER(?cap>0)
}
GROUP BY (?country)
}
#Labels
?country <http://schema.org/name> ?cname .
}
ORDER BY DESC(?avg)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q851'), ('cname', 'Saudi Arabia'), ('avg', '2458000000000')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('cname', "People's Republic of China"), ('avg', '486000000000')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('cname', 'Russia'), ('avg', '153619460512.5')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('cname', 'United States of America'), ('avg', '150952970833.333333333333333')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('cname', 'Soviet Union'), ('avg', '200804100')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('cname', 'Switzerland'), ('avg', '39000000')]


6

Compute min of total assests, max of total assets and average for each country.

In [45]:
#Q58
#repeat q57, q56, q55 for total assets
queryString = """
SELECT ?country ?cname ?min WHERE
{
{SELECT ?country (MIN(?assets) AS ?min)
WHERE { 

?s wdt:P2403 ?assets;
   wdt:P1454 wd:Q134161;
   wdt:P17 ?country.
   
FILTER(?assets>0)
}
GROUP BY (?country)
}
#Labels
?country <http://schema.org/name> ?cname .
}
ORDER BY ASC(?max)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('min', '74300000000')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('cname', "People's Republic of China"), ('min', '106214000000')]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('cname', 'Soviet Union'), ('min', '2446590000')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('cname', 'Russia'), ('min', '69064000')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('cname', 'Japan'), ('min', '1412070000')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('min', '1170110564.16')]
[('country', 'http://www.wikidata.org/entity/Q212'), ('cname', 'Ukraine'), ('min', '24463716000')]
[('country', 'http://www.wikidata.org/entity/Q213'), ('cname', 'Czech Republic'), ('min', '58464000')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('cname', 'Croatia'), ('min', '675233145000')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('cname', 'Hungary'), ('m

10

In [43]:
#Q59
#repeat q57, q56, q55 for total assets
queryString = """
SELECT ?country ?cname ?max WHERE
{
{SELECT ?country (MAX(?assets) AS ?max)
WHERE { 

?s wdt:P2403 ?assets;
   wdt:P1454 wd:Q134161;
   wdt:P17 ?country.
   
FILTER(?assets>0)
}
GROUP BY (?country)
}
#Labels
?country <http://schema.org/name> ?cname .
}
ORDER BY DESC(?max)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q884'), ('cname', 'South Korea'), ('max', '262174300000000')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('cname', 'Russia'), ('max', '6649579000000')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('cname', 'Hungary'), ('max', '4103786000000')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('cname', 'United States of America'), ('max', '3960490000000')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('cname', "People's Republic of China"), ('max', '2393844000000')]
[('country', 'http://www.wikidata.org/entity/Q851'), ('cname', 'Saudi Arabia'), ('max', '1494126000000')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('cname', 'Croatia'), ('max', '675233145000')]
[('country', 'http://www.wikidata.org/entity/Q878'), ('cname', 'United Arab Emirates'), ('max', '127587000000')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('max', '112400000000')]
[('country', 'http://www.wiki

10

In [42]:
#Q60
#repeat q57, q56, q55 for total assets
queryString = """
SELECT ?country ?cname ?avg WHERE
{
{SELECT ?country (AVG(?assets) AS ?avg)
WHERE { 

?s wdt:P2403 ?assets;
   wdt:P1454 wd:Q134161;
   wdt:P17 ?country.
   
FILTER(?assets>0)
}
GROUP BY (?country)
}
#Labels
?country <http://schema.org/name> ?cname .
}
ORDER BY DESC(?avg)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q884'), ('cname', 'South Korea'), ('avg', '88371322155240.666666666666667')]
[('country', 'http://www.wikidata.org/entity/Q28'), ('cname', 'Hungary'), ('avg', '2411893000000')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('cname', "People's Republic of China"), ('avg', '726726603035.373')]
[('country', 'http://www.wikidata.org/entity/Q224'), ('cname', 'Croatia'), ('avg', '675233145000')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('cname', 'Russia'), ('avg', '390861415222.222222222222222')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('cname', 'United States of America'), ('avg', '360837789813.400837837837838')]
[('country', 'http://www.wikidata.org/entity/Q878'), ('cname', 'United Arab Emirates'), ('avg', '82935411333.333333333333333')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('avg', '74300000000')]
[('country', 'http://www.wikidata.org/entity/Q851'), ('cname', 'Saudi Arab

10

<h3> TASK 5.2 ENDS HERE </h3>
I found the properties about Market Cap, Net Profit, Total Assets and Total revenue. I computed a set of queries to find out for some countries
that is the min, max and average market cap and total assets for a company of type jonit-stock. As a result, China is the leader for what concerns
market cap, while South Korea is on top in the total assets rankings.

<h3> TASK 5.3 STARTS HERE </h3>
Which business in each country owns more businesses in other EU. countries?

I am trying to reuse that i found out for task 3.

In [49]:
#Q61
#try to reuse that i found out for task 3
queryString = """
SELECT ?s ?sname ?OwnedCompanies WHERE
{
{SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o.
?o wdt:P31 ?type;
   wdt:P17 ?c.
?c wdt:P463 wd:Q458.

FILTER( ( (?type=wd:Q6881511)||(?type=wd:Q4830453) ) && (?country!=?c) ) 
}
GROUP BY (?s)

}

#Labels
?s <http://schema.org/name> ?sname .
}
ORDER BY DESC(?OwnedCompanies)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q292750'), ('sname', 'AG für Verkehrswesen'), ('OwnedCompanies', '23')]
[('s', 'http://www.wikidata.org/entity/Q94639662'), ('sname', 'NexGen'), ('OwnedCompanies', '13')]
[('s', 'http://www.wikidata.org/entity/Q180855'), ('sname', 'Heineken'), ('OwnedCompanies', '7')]
[('s', 'http://www.wikidata.org/entity/Q504998'), ('sname', 'LVMH'), ('OwnedCompanies', '7')]
[('s', 'http://www.wikidata.org/entity/Q77846713'), ('sname', 'Veolia Central & Eastern Europe'), ('OwnedCompanies', '7')]
[('s', 'http://www.wikidata.org/entity/Q150437'), ('sname', 'Penta Investments Limited'), ('OwnedCompanies', '7')]
[('s', 'http://www.wikidata.org/entity/Q499707'), ('sname', 'BNP Paribas'), ('OwnedCompanies', '6')]
[('s', 'http://www.wikidata.org/entity/Q300704'), ('sname', 'Aachen Münchener'), ('OwnedCompanies', '6')]
[('s', 'http://www.wikidata.org/entity/Q48916492'), ('sname', 'E.ON Beteiligungen'), ('OwnedCompanies', '5')]
[('s', 'http://www.wikidata.org/ent

10

In [52]:
#Q62
#add country
queryString = """
SELECT ?s ?country ?OwnedCompanies WHERE
{
  ?s wdt:P17 ?country.
{
SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o.
?o wdt:P31 ?type;
   wdt:P17 ?c.
?c wdt:P463 wd:Q458.

FILTER( ( (?type=wd:Q6881511)||(?type=wd:Q4830453) ) && (?country!=?c) ) 
}
GROUP BY (?s)
}
}
ORDER BY DESC(?OwnedCompanies)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q292750'), ('country', 'http://www.wikidata.org/entity/Q1206012'), ('OwnedCompanies', '23')]
[('s', 'http://www.wikidata.org/entity/Q94639662'), ('country', 'http://www.wikidata.org/entity/Q32'), ('OwnedCompanies', '13')]
[('s', 'http://www.wikidata.org/entity/Q180855'), ('country', 'http://www.wikidata.org/entity/Q55'), ('OwnedCompanies', '7')]
[('s', 'http://www.wikidata.org/entity/Q504998'), ('country', 'http://www.wikidata.org/entity/Q142'), ('OwnedCompanies', '7')]
[('s', 'http://www.wikidata.org/entity/Q77846713'), ('country', 'http://www.wikidata.org/entity/Q142'), ('OwnedCompanies', '7')]
[('s', 'http://www.wikidata.org/entity/Q150437'), ('country', 'http://www.wikidata.org/entity/Q785'), ('OwnedCompanies', '7')]
[('s', 'http://www.wikidata.org/entity/Q499707'), ('country', 'http://www.wikidata.org/entity/Q142'), ('OwnedCompanies', '6')]
[('s', 'http://www.wikidata.org/entity/Q300704'), ('country', 'http://www.wikidata.org/entity/Q

10

In [54]:
#TASK 5.3 ENDS HERE:
queryString = """
SELECT ?country (MAX (?OwnedCompanies) AS ?max) WHERE
{
{SELECT ?s ?country ?OwnedCompanies WHERE
{
  ?s wdt:P17 ?country.
{
SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o.
?o wdt:P31 ?type;
   wdt:P17 ?c.
?c wdt:P463 wd:Q458.

FILTER( ( (?type=wd:Q6881511)||(?type=wd:Q4830453) ) && (?country!=?c) ) 
}
GROUP BY (?s)
}
}}
}
GROUP BY (?country)
ORDER BY DESC(?max)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('max', '23')]
[('country', 'http://www.wikidata.org/entity/Q32'), ('max', '13')]
[('country', 'http://www.wikidata.org/entity/Q785'), ('max', '7')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('max', '7')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('max', '7')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('max', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('max', '5')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('max', '5')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('max', '4')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('max', '4')]


10

<h3> TASK 5.3 ENDS HERE </h3>
Computed for each EU countries the max number of businesses owneed and then retrieve the top 10.

In [57]:
#TASK 5.3 ENDS HERE
#add s labels
queryString = """
SELECT (GROUP_CONCAT(DISTINCT ?s; separator=", ") AS ?Companies) ?countries ?OwnedCompanies WHERE
{
  ?s wdt:P17 ?countries.
{
SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o.
?o wdt:P31 ?type;
   wdt:P17 ?c.
?c wdt:P463 wd:Q458.

FILTER( ( (?type=wd:Q6881511)||(?type=wd:Q4830453) ) && (?country!=?c) ) 
}
GROUP BY (?s)
}

{
SELECT ?country (MAX (?OwnedCompanies) AS ?max) WHERE
{
{SELECT ?s ?country ?OwnedCompanies WHERE
{
  ?s wdt:P17 ?country.
{
SELECT ?s (COUNT(DISTINCT (?o)) AS ?OwnedCompanies)
WHERE { 

?s wdt:P31 ?type;
   wdt:P17 ?country;
   wdt:P1830 ?o.
?o wdt:P31 ?type;
   wdt:P17 ?c.
?c wdt:P463 wd:Q458.

FILTER( ( (?type=wd:Q6881511)||(?type=wd:Q4830453) ) && (?country!=?c) ) 
}
GROUP BY (?s)
}
}}
}
GROUP BY (?country)
}
FILTER( (?OwnedCompanies=?max) && (?countries=?country) ) 
}
GROUP BY ?countries ?OwnedCompanies
ORDER BY DESC(?OwnedCompanies)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('Companies', 'http://www.wikidata.org/entity/Q292750'), ('countries', 'http://www.wikidata.org/entity/Q1206012'), ('OwnedCompanies', '23')]
[('Companies', 'http://www.wikidata.org/entity/Q94639662'), ('countries', 'http://www.wikidata.org/entity/Q32'), ('OwnedCompanies', '13')]
[('Companies', 'http://www.wikidata.org/entity/Q150437'), ('countries', 'http://www.wikidata.org/entity/Q785'), ('OwnedCompanies', '7')]
[('Companies', 'http://www.wikidata.org/entity/Q504998, http://www.wikidata.org/entity/Q77846713'), ('countries', 'http://www.wikidata.org/entity/Q142'), ('OwnedCompanies', '7')]
[('Companies', 'http://www.wikidata.org/entity/Q180855'), ('countries', 'http://www.wikidata.org/entity/Q55'), ('OwnedCompanies', '7')]
[('Companies', 'http://www.wikidata.org/entity/Q300704'), ('countries', 'http://www.wikidata.org/entity/Q183'), ('OwnedCompanies', '6')]
[('Companies', 'http://www.wikidata.org/entity/Q122141'), ('countries', 'http://www.wikidata.org/entity/Q145'), ('OwnedCom

10

<h3> TASK 5.4 STARTS HERE </h3>
What can we say about industry sectors in various countries?

From task 1 i know P452 is the property related to industry, check how many industry types there are in France.

In [59]:
#Q63
#from task 1 i know P452 is the property related to industry, check how many industry types there are in france
queryString = """
SELECT ?industry ?iname ?howMany WHERE{
{
SELECT ?industry (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P452 ?industry;
   wdt:P17 wd:Q142.
}
GROUP BY (?industry)
}
#Labels
?industry <http://schema.org/name> ?iname .
}
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('industry', 'http://www.wikidata.org/entity/Q190117'), ('iname', 'automotive industry'), ('howMany', '183')]
[('industry', 'http://www.wikidata.org/entity/Q941594'), ('iname', 'video game industry'), ('howMany', '105')]
[('industry', 'http://www.wikidata.org/entity/Q29584334'), ('iname', 'Financial service activities, except insurance and pension funding'), ('howMany', '101')]
[('industry', 'http://www.wikidata.org/entity/Q126793'), ('iname', 'retail'), ('howMany', '85')]
[('industry', 'http://www.wikidata.org/entity/Q1924906'), ('iname', 'metal industry'), ('howMany', '75')]
[('industry', 'http://www.wikidata.org/entity/Q44497'), ('iname', 'mining'), ('howMany', '69')]
[('industry', 'http://www.wikidata.org/entity/Q43183'), ('iname', 'insurance'), ('howMany', '69')]
[('industry', 'http://www.wikidata.org/entity/Q107601756'), ('iname', 'food and tobacco industry'), ('howMany', '60')]
[('industry', 'http://www.wikidata.org/entity/Q29584657'), ('iname', 'Electric power generati

10

Retrieve the most common industry in EU.

In [60]:
#Q64
#retriev the most common industry in EU
queryString = """
SELECT ?industry ?iname ?howMany WHERE{
{
SELECT ?industry (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P452 ?industry;
   wdt:P17 ?country.
?country wdt:P463 wd:Q458.
}
GROUP BY (?industry)
}
#Labels
?industry <http://schema.org/name> ?iname .
}
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('industry', 'http://www.wikidata.org/entity/Q869095'), ('iname', 'brewing'), ('howMany', '1357')]
[('industry', 'http://www.wikidata.org/entity/Q190117'), ('iname', 'automotive industry'), ('howMany', '1325')]
[('industry', 'http://www.wikidata.org/entity/Q941594'), ('iname', 'video game industry'), ('howMany', '1005')]
[('industry', 'http://www.wikidata.org/entity/Q29584334'), ('iname', 'Financial service activities, except insurance and pension funding'), ('howMany', '870')]
[('industry', 'http://www.wikidata.org/entity/Q126793'), ('iname', 'retail'), ('howMany', '802')]
[('industry', 'http://www.wikidata.org/entity/Q15067276'), ('iname', 'health care industry'), ('howMany', '653')]
[('industry', 'http://www.wikidata.org/entity/Q107601756'), ('iname', 'food and tobacco industry'), ('howMany', '573')]
[('industry', 'http://www.wikidata.org/entity/Q1283714'), ('iname', 'Crop production'), ('howMany', '539')]
[('industry', 'http://www.wikidata.org/entity/Q43183'), ('iname', 'i

10

Retrive the top 10 industry sectors for average net profit in EU.

In [61]:
#Q65
#retrive the top 10 industry sectors for average net profit in EU
queryString = """
SELECT ?industry ?iname ?avgNet WHERE{
{
SELECT ?industry (AVG(DISTINCT (?net)) AS ?avgNet)
WHERE { 

?s wdt:P452 ?industry;
   wdt:P2226 ?net; 
   wdt:P17 ?country.
?country wdt:P463 wd:Q458.
}
GROUP BY (?industry)
}
#Labels
?industry <http://schema.org/name> ?iname .
}
ORDER BY DESC(?avgNet)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('industry', 'http://www.wikidata.org/entity/Q2151621'), ('iname', 'energy industry'), ('avgNet', '238400000000')]
[('industry', 'http://www.wikidata.org/entity/Q29584334'), ('iname', 'Financial service activities, except insurance and pension funding'), ('avgNet', '206133333333.333333333333333')]
[('industry', 'http://www.wikidata.org/entity/Q1416657'), ('iname', 'Finance'), ('avgNet', '200025000000')]
[('industry', 'http://www.wikidata.org/entity/Q806718'), ('iname', 'banking industry'), ('avgNet', '200025000000')]
[('industry', 'http://www.wikidata.org/entity/Q11379'), ('iname', 'energy'), ('avgNet', '139000000000')]
[('industry', 'http://www.wikidata.org/entity/Q12089268'), ('iname', 'Extraction of crude petroleum and natural gas'), ('avgNet', '139000000000')]
[('industry', 'http://www.wikidata.org/entity/Q43183'), ('iname', 'insurance'), ('avgNet', '138244846881')]
[('industry', 'http://www.wikidata.org/entity/Q837171'), ('iname', 'financial services'), ('avgNet', '136598

10

Same as above but for total revenue.

In [68]:
#Q66
#same as above but for total revenue
queryString = """
SELECT ?industry ?iname ?avgRev WHERE{
{
SELECT ?industry (AVG (?revenue) AS ?avgRev)
WHERE { 

?s wdt:P452 ?industry;
   wdt:P2139 ?revenue;
   wdt:P17 ?country.
?country wdt:P463 wd:Q458.
}
GROUP BY (?industry)
}
#Labels
?industry <http://schema.org/name> ?iname .
}
ORDER BY DESC(?avgRev)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('industry', 'http://www.wikidata.org/entity/Q56604526'), ('iname', 'oil industry'), ('avgRev', '344877000000')]
[('industry', 'http://www.wikidata.org/entity/Q12089268'), ('iname', 'Extraction of crude petroleum and natural gas'), ('avgRev', '257938500000')]
[('industry', 'http://www.wikidata.org/entity/Q862571'), ('iname', 'petroleum industry'), ('avgRev', '224345496285.714285714285714')]
[('industry', 'http://www.wikidata.org/entity/Q778575'), ('iname', 'conglomerate'), ('avgRev', '189580477000')]
[('industry', 'http://www.wikidata.org/entity/Q29584629'), ('iname', 'Manufacture of motor vehicles'), ('avgRev', '153261000000')]
[('industry', 'http://www.wikidata.org/entity/Q1867977'), ('iname', 'refinery'), ('avgRev', '130768000000')]
[('industry', 'http://www.wikidata.org/entity/Q11379'), ('iname', 'energy'), ('avgRev', '88033171500')]
[('industry', 'http://www.wikidata.org/entity/Q17177506'), ('iname', 'lighting technic'), ('avgRev', '83000000000')]
[('industry', 'http://ww

10

Check countries with more oil industry in EU.

In [63]:
#Q67
#check countries with more oil industry in EU
queryString = """
SELECT ?country ?cname ?howMany WHERE{
{
SELECT ?country (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P452 wd:Q56604526;
   wdt:P17 ?country.
?country wdt:P463 wd:Q458.
}
GROUP BY (?country)
}
#Labels
?country <http://schema.org/name> ?cname .
}
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('cname', 'United Kingdom'), ('howMany', '2')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('cname', 'Kingdom of the Netherlands'), ('howMany', '1')]


2

In [64]:
#Q68
#check countries with more oil industries
queryString = """
SELECT ?country ?cname ?howMany WHERE{
{
SELECT ?country (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P452 wd:Q56604526;
   wdt:P17 ?country.
}
GROUP BY (?country)
}
#Labels
?country <http://schema.org/name> ?cname .
}
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('cname', 'United Kingdom'), ('howMany', '2')]
[('country', 'http://www.wikidata.org/entity/Q174193'), ('cname', 'United Kingdom of Great Britain and Ireland'), ('howMany', '1')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('cname', 'Kingdom of the Netherlands'), ('howMany', '1')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('cname', 'Netherlands'), ('howMany', '1')]


4

Compute the revenue of english oil industries.

In [66]:
#Q69
queryString = """
SELECT ?s ?sname ?rev WHERE{

?s wdt:P452 wd:Q56604526;
   wdt:P2139 ?rev;
   wdt:P17 wd:Q145.
   
#Labels
?s <http://schema.org/name> ?sname .
}
ORDER BY DESC(?rev)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q154950'), ('sname', 'Royal Dutch Shell'), ('rev', '344877000000')]


1

From the name it seems to be dutch, run the same query for dutch oil company.

In [67]:
#Q70
queryString = """
SELECT ?s ?sname ?rev WHERE{

?s wdt:P452 wd:Q56604526;
   wdt:P2139 ?rev;
   wdt:P17 wd:Q55.
   
#Labels
?s <http://schema.org/name> ?sname .
}
ORDER BY DESC(?rev)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q154950'), ('sname', 'Royal Dutch Shell'), ('rev', '344877000000')]


1

Same as Q66 but summing total revenue.

In [70]:
#Q71
#same as Q66 but summing total revenue
queryString = """
SELECT ?industry ?iname ?TotalRev WHERE{
{
SELECT ?industry (SUM (?revenue) AS ?TotalRev)
WHERE { 

?s wdt:P452 ?industry;
   wdt:P2139 ?revenue;
   wdt:P17 ?country.
?country wdt:P463 wd:Q458.
}
GROUP BY (?industry)
}
#Labels
?industry <http://schema.org/name> ?iname .
}
ORDER BY DESC(?TotalRev)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('industry', 'http://www.wikidata.org/entity/Q862571'), ('iname', 'petroleum industry'), ('TotalRev', '4711255422000')]
[('industry', 'http://www.wikidata.org/entity/Q190117'), ('iname', 'automotive industry'), ('TotalRev', '1846775398310.37')]
[('industry', 'http://www.wikidata.org/entity/Q126793'), ('iname', 'retail'), ('TotalRev', '1180207627743')]
[('industry', 'http://www.wikidata.org/entity/Q2285982'), ('iname', 'ferrous metallurgy'), ('TotalRev', '658094835000')]
[('industry', 'http://www.wikidata.org/entity/Q207652'), ('iname', 'chemical industry'), ('TotalRev', '647576552000')]
[('industry', 'http://www.wikidata.org/entity/Q25245117'), ('iname', 'telecommunications industry'), ('TotalRev', '595121274852')]
[('industry', 'http://www.wikidata.org/entity/Q778575'), ('iname', 'conglomerate'), ('TotalRev', '568741431000')]
[('industry', 'http://www.wikidata.org/entity/Q2151621'), ('iname', 'energy industry'), ('TotalRev', '539069864380')]
[('industry', 'http://www.wikidata

10

Check countries with more petroleum industries.

In [71]:
#Q72
#check countries with more petroleum industries
queryString = """
SELECT ?country ?cname ?howMany WHERE{
{
SELECT ?country (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P452 wd:Q862571;
   wdt:P17 ?country.
}
GROUP BY (?country)
}
#Labels
?country <http://schema.org/name> ?cname .
}
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('cname', 'United States of America'), ('howMany', '124')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('cname', 'United Kingdom'), ('howMany', '64')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('howMany', '33')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('cname', 'Canada'), ('howMany', '31')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('cname', 'India'), ('howMany', '23')]
[('country', 'http://www.wikidata.org/entity/Q20'), ('cname', 'Norway'), ('howMany', '21')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('cname', 'Russia'), ('howMany', '18')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('howMany', '16')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('cname', "People's Republic of China"), ('howMany', '16')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('cname', 'Brazil'), ('howMany', '16')]


10

Check countries with more automotive industries.

In [72]:
#Q73
#check countries with more automotive industries
queryString = """
SELECT ?country ?cname ?howMany WHERE{
{
SELECT ?country (COUNT(DISTINCT (?s)) AS ?howMany)
WHERE { 

?s wdt:P452 wd:Q190117;
   wdt:P17 ?country.
}
GROUP BY (?country)
}
#Labels
?country <http://schema.org/name> ?cname .
}
ORDER BY DESC(?howMany)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany'), ('howMany', '374')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('cname', 'United States of America'), ('howMany', '321')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('cname', 'United Kingdom'), ('howMany', '312')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('cname', 'France'), ('howMany', '183')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('cname', 'Italy'), ('howMany', '175')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('cname', 'Spain'), ('howMany', '124')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('cname', "People's Republic of China"), ('howMany', '89')]
[('country', 'http://www.wikidata.org/entity/Q43287'), ('cname', 'German Empire'), ('howMany', '73')]
[('country', 'http://www.wikidata.org/entity/Q41304'), ('cname', 'Weimar Republic'), ('howMany', '64')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('cname', 'Japan'), ('howMany', '53

10

Retrieve top industry sectors for italy by revenue.

In [74]:
#Q74
#check top industry sectors for italy
queryString = """
SELECT ?industry ?iname ?sumRev WHERE{
{
SELECT ?industry (SUM (?revenue) AS ?sumRev)
WHERE { 

?s wdt:P452 ?industry;
   wdt:P2139 ?revenue;
   wdt:P17 wd:Q38.
}
GROUP BY (?industry)
}
#Labels
?industry <http://schema.org/name> ?iname .
}
ORDER BY DESC(?sumRev)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('industry', 'http://www.wikidata.org/entity/Q2518196'), ('iname', 'insurance industry'), ('sumRev', '73188000000')]
[('industry', 'http://www.wikidata.org/entity/Q862571'), ('iname', 'petroleum industry'), ('sumRev', '43987000000')]
[('industry', 'http://www.wikidata.org/entity/Q188460'), ('iname', 'natural resource'), ('sumRev', '43987000000')]
[('industry', 'http://www.wikidata.org/entity/Q11467'), ('iname', 'metallurgy'), ('sumRev', '25477000000')]
[('industry', 'http://www.wikidata.org/entity/Q190117'), ('iname', 'automotive industry'), ('sumRev', '14739401000')]
[('industry', 'http://www.wikidata.org/entity/Q2876213'), ('iname', 'aerospace'), ('sumRev', '13640000000')]
[('industry', 'http://www.wikidata.org/entity/Q540912'), ('iname', 'food industry'), ('sumRev', '11526671062')]
[('industry', 'http://www.wikidata.org/entity/Q177777'), ('iname', 'logistics'), ('sumRev', '9402965143')]
[('industry', 'http://www.wikidata.org/entity/Q107598157'), ('iname', 'plastics and rubb

10

<h3> TASK 5.4 ENDS HERE </h3>
I found out that the leading industry is the one about energy, especially oil and petrolium, as we can see from the Q71 results. On top of that
the production is mainly in USA and England. Italy is in fifth place for amount of revenues incoming from automotive, meanwhile its most
productive sector is the insurance one as we can see from Q74.